# Tugas Scraping

# Scraping liputan6.com
Pada python notebook ini akan dijelaskan mengenai scraping berita liputan6.com/news Scraping dimulai dari halaman awal, lalu akan menelusuri setiap link, dan akan mengambil berita beserta properties lainnya.

## Inisialisasi urllib.request

In [1]:
import urllib.request

r = urllib.request.urlopen('https://www.liputan6.com/news').read()
print(type(r))
print(r[1:1000])

<class 'bytes'>
b'!DOCTYPE html><html itemscope="itemscope" itemtype="http://schema.org/WebPage"><head><title>Berita Politik, Hukum, Dunia International - Indonesia News Today</title><meta charset="utf-8"><meta name="viewport" content="width=1024"><meta name="adx:sections" content="news"><meta http-equiv="refresh" content="900"><meta name="title" content="Berita Politik, Hukum, Dunia International - Indonesia News Today"><meta name="description" content="Liputan6.com Indonesia News Today, menyajikan kabar berita terkini indonesia dan dunia internasional meliputi berita politik hingga hukum dan kriminal"><meta name="keywords" content="news today, berita internasional, berita dunia, berita politik, berita kriminal"><meta content="index,follow" name="googlebot-news">\n<meta content="index,follow" name="googlebot">\n<meta name="robots" content="index,follow">\n<meta content="Liputan6.com" name="author">\n<meta content="id" name="language">\n<meta content="id" name="geo.country">\n<meta htt

## Inisialisasi BeautifulSoup

In [2]:
from bs4 import BeautifulSoup 

soup = BeautifulSoup(r,"lxml")
print(type(soup))
print(soup.prettify()[1:1000])

<class 'bs4.BeautifulSoup'>
!DOCTYPE html>
<html itemscope="itemscope" itemtype="http://schema.org/WebPage">
 <head>
  <title>
   Berita Politik, Hukum, Dunia International - Indonesia News Today
  </title>
  <meta charset="utf-8"/>
  <meta content="width=1024" name="viewport"/>
  <meta content="news" name="adx:sections"/>
  <meta content="900" http-equiv="refresh"/>
  <meta content="Berita Politik, Hukum, Dunia International - Indonesia News Today" name="title"/>
  <meta content="Liputan6.com Indonesia News Today, menyajikan kabar berita terkini indonesia dan dunia internasional meliputi berita politik hingga hukum dan kriminal" name="description"/>
  <meta content="news today, berita internasional, berita dunia, berita politik, berita kriminal" name="keywords"/>
  <meta content="index,follow" name="googlebot-news"/>
  <meta content="index,follow" name="googlebot"/>
  <meta content="index,follow" name="robots"/>
  <meta content="Liputan6.com" name="author"/>
  <meta content="id" name=

## Dapatkan Semua Link

In [3]:
links = soup.find_all('a')
print(len(links))

for link in links:
    try:
        print(link.text.strip(),link['href'])
    except:
        pass

288
liputan6 https://www.liputan6.com
 https://www.liputan6.com
Home https://www.liputan6.com
News https://www.liputan6.com/news
Bisnis https://www.liputan6.com/bisnis
ShowBiz https://www.liputan6.com/showbiz
Bola https://www.liputan6.com/bola
Foto https://www.liputan6.com/photo
Tekno https://www.liputan6.com/tekno
Cek Fakta https://www.liputan6.com/cek-fakta
Video https://www.liputan6.com/video
Hot https://hot.liputan6.com
Disabilitas https://www.liputan6.com/disabilitas
Global https://www.liputan6.com/global
Otomotif https://www.liputan6.com/otomotif
On Off https://www.liputan6.com/on-off
Surabaya https://surabaya.liputan6.com
Regional https://www.liputan6.com/regional
Lifestyle https://www.liputan6.com/lifestyle
Health https://www.liputan6.com/health
Citizen6 https://www.liputan6.com/citizen6
TV https://www.vidio.com/live
Properti https://www.liputan6.com/properti
News https://www.liputan6.com/news
 #
Politik https://www.liputan6.com/news/politik
Peristiwa https://www.liputan6.com/n

Link yang benar-benar menuju ke berita adalah link yang mengandung '/read/' dan benar-benar link bukan javascript ,link '#',link facebook.com, link plus.google maupun link twitter.com sehingga perlu difilter sebagai berikut:

In [4]:
list_key_link = ['/read/']
berita_link = []
for i in list_key_link:
    berita_link = list(set(berita_link)) + list(set([link for link in links if i in str(link) and 'javascript:void(0)' not in str(link) 
               and 'twitter.com' not in str(link) and 'facebook.com' not in str(link) and 'plus.google.com' not in str(link)]))
print(len(berita_link))
for link in berita_link:
    print('%s: %s'%(link.text.strip(),link["href"]))

177
01:27: https://www.liputan6.com/news/read/4128655/uji-coba-sistem-2-1-arah-puncak-bogor-mengular-sepanjang-5-km
Live Streaming Dear Netizen: Membaca Arah Golkar di Tangan Airlangga: https://www.liputan6.com/news/read/4127777/live-streaming-dear-netizen-membaca-arah-golkar-di-tangan-airlangga?HouseAds&campaign=Golkar_News_STS3
: https://www.liputan6.com/news/read/4127396/airlangga-hartarto-kembali-pimpin-partai-golkar?HouseAds&campaign=Golkar_News_STS3
: https://www.liputan6.com/news/read/4128676/usai-kasus-harley-garuda-indonesia-diisukan-angkut-ferrari-ilegal-ini-faktanya
10 Tahun Disimpan, Mortir Temuan Warga Palopo Diledakkan Polda Sulsel: https://www.liputan6.com/news/read/4128565/10-tahun-disimpan-mortir-temuan-warga-palopo-diledakkan-polda-sulsel
Wanita Tua di Jember Jadi Korban Kekerasan Seksual di Rumah Majikan: https://www.liputan6.com/news/read/4128561/wanita-tua-di-jember-jadi-korban-kekerasan-seksual-di-rumah-majikan
: https://www.liputan6.com/news/read/4128636/kerja-fl

## Dapatkan Isi Berita
Di sini akan diambil judul berita, isi berita, dan tanggal berita dipost/dibuat. Dari hasil inspeksi, judul ada pada elemen h1 dengan class read-page--header--title entry-title . Untuk isi berita ada pada elemen div class read-content--content. Sedangkan untuk tanggal ada pada elemen time class read-page--header--author__datetime updated.

In [5]:
html = urllib.request.urlopen("https://www.liputan6.com/news/read/3678213/jokowi-minta-timnas-indonesia-u-19-berikan-yang-terbaik-saat-lawan-jepang").read()
soup = BeautifulSoup(html, "lxml")

judul = soup.find("h1","read-page--header--title entry-title").text.strip()
#mengambil semua text dari div body
isi=''
isi_berita = soup.find('div','read-page--content')
for x in isi_berita.find_all('p'):
    isi=isi+x.text.strip()
tanggal = soup.find("time","read-page--header--author__datetime updated").text[0:11]

print(tanggal)
print(judul)
print(isi)


28 Okt 2018
Jokowi Minta Timnas Indonesia U-19 Berikan yang Terbaik Saat Lawan Jepang
Liputan6.com, Jakarta - Presiden Joko Widodo atau Jokowi berharap, Timnas Indonesia U-19 menang melawan Jepang pada babak 8 besar Piala AFC U-19 2018. Laga sepakbola ini digelar di Stadion Utama Gelora Bung Karno, Senayan, Jakarta, malam nanti."Menang. Harapan kita skornya menang, angkanya terserah yang penting menang," kata Jokowi di Pendopo Delta Wibawa Sidoarjo, Jawa Timur, Minggu (28/10/2018).Baca JugaJokowi mengaku akan menyaksikan pertandingan Timnas Indonesia U-19 melawan Jepang melalui layar televisi. Dia tak bisa menyaksikan pertandingan secara langsung di GBK. "Nonton lewat TV saja," ujar dia.Jokowi berharap, Timnas Indonesia U-19 optimis untuk menang. Para pemain juga harus tetap semangat dan pantang menyerah."Pemuda U-19 harus optimis berikan yang terbaik untuk bangsa Indonesia, rakyat Indonesia," Jokowi menandaskan.Timnas Indonesia U-19 bertemu Jepang dalam upaya mewujudkan target lolos k

Menyimpan gambar dari berita, gambar ada pada div class photo

In [7]:
from PIL import Image
from io import BytesIO
import string

gambar = soup.find("div","read-page--top-media").find("img","read-page--photo-gallery--item__picture-lazyload")
gambar['data-src']
# print(gambar['data-src'])

r = urllib.request.urlopen(gambar["data-src"]).read()
# print(r)

i = Image.open(BytesIO(r))
exclude = set(string.punctuation)
judul = ''.join(str(ch) for ch in judul if ch not in exclude)

nama_file = "gambar/" + judul + ".png"
i.save(nama_file,'PNG')

In [11]:
# berita = []
# for link in links1:
#     print('%s: %s'%(link.text.strip(),link["href"]))
#     try:
#         berita.append(getBerita(link["href"]))
#     except:
#         pass

## Fungsi Scraping
merupakan fungsi scraping yang utuh untuk scraping berita dari liputan6.com/news

In [8]:
def getBerita(link):
    
    html = urllib.request.urlopen(link).read()
    soup = BeautifulSoup(html, "lxml")
    #mengambil judul
    judul = soup.find("h1","read-page--header--title entry-title").text.strip()
    #mengambil semua text dari div read-page--content
    isi=''
    isi_berita = soup.find('div','read-page--content')
    for x in isi_berita.find_all('p'):
        isi=isi+x.text.strip()
    #mengambil tanggal
    tanggal = soup.find("time","read-page--header--author__datetime updated").text[0:11]    
    
    #mengambil gambar
    from PIL import Image
    from io import BytesIO
    import string

    gambar = soup.find("div","read-page--top-media").find("img","read-page--photo-gallery--item__picture-lazyload")
    gambar['data-src']
    # print(gambar['data-src'])

    r = urllib.request.urlopen(gambar["data-src"]).read()
    # print(r)

    i = Image.open(BytesIO(r))
    exclude = set(string.punctuation)
    judul = ''.join(str(ch) for ch in judul if ch not in exclude)

    nama_file = "gambar/" + judul + ".png"
    i.save(nama_file,'PNG')
    
    return [tanggal,judul,isi]

## Dapatkan semua data dari link berita

In [10]:
# getBerita('https://ekonomi.kompas.com/read/2018/10/24/204900026/pertamina-ingin-beli-minyak-dengan-mata-uang-selain-dollar-as')
berita = []
for link in berita_link:
    print('%s: %s'%(link.text.strip(),link["href"]))
    try:
        berita.append(getBerita(link["href"]))
    except:
        pass

01:27: https://www.liputan6.com/news/read/4128655/uji-coba-sistem-2-1-arah-puncak-bogor-mengular-sepanjang-5-km
Live Streaming Dear Netizen: Membaca Arah Golkar di Tangan Airlangga: https://www.liputan6.com/news/read/4127777/live-streaming-dear-netizen-membaca-arah-golkar-di-tangan-airlangga?HouseAds&campaign=Golkar_News_STS3
: https://www.liputan6.com/news/read/4127396/airlangga-hartarto-kembali-pimpin-partai-golkar?HouseAds&campaign=Golkar_News_STS3
: https://www.liputan6.com/news/read/4128676/usai-kasus-harley-garuda-indonesia-diisukan-angkut-ferrari-ilegal-ini-faktanya
10 Tahun Disimpan, Mortir Temuan Warga Palopo Diledakkan Polda Sulsel: https://www.liputan6.com/news/read/4128565/10-tahun-disimpan-mortir-temuan-warga-palopo-diledakkan-polda-sulsel
Wanita Tua di Jember Jadi Korban Kekerasan Seksual di Rumah Majikan: https://www.liputan6.com/news/read/4128561/wanita-tua-di-jember-jadi-korban-kekerasan-seksual-di-rumah-majikan
: https://www.liputan6.com/news/read/4128636/kerja-fleksi

: https://www.liputan6.com/news/read/4126664/lahirnya-provinsi-gorontalo-bumi-serambi-madinah-19-tahun-lalu?HouseAds&campaign=SEjarahHariIni_News_STS1
: https://www.liputan6.com/news/read/4128794/polda-jatim-gerebek-gudang-pembuatan-senjata-api-ilegal-di-lumajang?medium=Headline&campaign=Headline_click_1
01:18: https://www.liputan6.com/news/read/4128477/detik-detik-derasnya-banjir-seret-kendaraan-warga-di-bandung
Bocah 12 Tahun Hanyut di Aliran Sungai Ciliwung, Bogor: https://www.liputan6.com/news/read/4128639/bocah-12-tahun-hanyut-di-aliran-sungai-ciliwung-bogor
: https://www.liputan6.com/news/read/4128371/ahy-buka-pendidikan-tahap-akhir-akademi-demokrat-angkatan-pertama
: https://www.liputan6.com/news/read/4128601/kpk-kecewa-internal-garuda-indonesia-masih-bobrok
Erick Thohir: Penyelundupan Harley Davidson Dilakukan Secara Sistemik: https://www.liputan6.com/news/read/4128662/erick-thohir-penyelundupan-harley-davidson-dilakukan-secara-sistemik
+4: https://www.liputan6.com/news/read/41

Top 3 News: Menanti Sanksi Truk Brimob Lawan Arus di Jalur Transjakarta: https://www.liputan6.com/news/read/4128448/top-3-news-menanti-sanksi-truk-brimob-lawan-arus-di-jalur-transjakarta
: https://www.liputan6.com/news/read/4127777/live-streaming-dear-netizen-membaca-arah-golkar-di-tangan-airlangga?HouseAds&campaign=Golkar_News_STS3
VIDEO: Jalur Puncak, Polisi Terapkan Sistem Kanal 2-1: https://www.liputan6.com/news/read/4128783/video-jalur-puncak-polisi-terapkan-sistem-kanal-2-1
: https://www.liputan6.com/news/read/4128654/polisi-truk-brimob-lawan-arus-di-busway-kantongi-izin
AHY Buka Pendidikan Tahap Akhir Akademi Demokrat Angkatan Pertama: https://www.liputan6.com/news/read/4128371/ahy-buka-pendidikan-tahap-akhir-akademi-demokrat-angkatan-pertama
Ada Bangku Taman, Begini Penampakan Trotoar Cikini: https://www.liputan6.com/news/read/4128684/ada-bangku-taman-begini-penampakan-trotoar-cikini
Listyo Sigit Janji Akan Amanah Jadi Kabareskrim: https://www.liputan6.com/news/read/4128440/lis

07 Des 2019 15:36Rakernas PAN Ricuh, Amien Rais Istighfar saat Lihat Kader Saling Dorong: https://www.liputan6.com/news/read/4128743/rakernas-pan-ricuh-amien-rais-istighfar-saat-lihat-kader-saling-dorong
Detik-Detik Derasnya Banjir Seret Kendaraan Warga di Bandung: https://www.liputan6.com/news/read/4128477/detik-detik-derasnya-banjir-seret-kendaraan-warga-di-bandung
: https://www.liputan6.com/news/read/4128694/diterjang-banjir-bandang-jembatan-di-bogor-ini-tinggal-sepotong


In [11]:
import pandas as pd
df =pd.DataFrame(berita, columns=['tanggal','judul','isi'])
df

,tanggal,judul,isi
0,06 Des 2019,Live Streaming Dear Netizen Membaca Arah Golka...,"Liputan6.com, Jakarta - Airlangga Hartarto ter..."
1,06 Des 2019,Airlangga Hartarto Kembali Pimpin Partai Golkar,"Liputan6.com, Jakarta - Airlangga Hartarto ter..."
2,07 Des 2019,Usai Kasus Harley Garuda Indonesia Diisukan An...,"Liputan6.com, Jakarta - PT Garuda Indonesia an..."
3,07 Des 2019,Kerja Fleksibel ASN Akan Diuji Coba Mulai 1 Ja...,"Liputan6.com, Jakarta - Pemerintah berencana m..."
4,07 Des 2019,Dapat Anugerah Asian of The Year Ini Reaksi Jo...,"Liputan6.com, Jakarta - Presiden Jokowi menang..."
...,...,...,...
132,07 Des 2019,Ketum ProJo Pak Jokowi Tak Kasih Posisi Wamenh...,"Liputan6.com, Jakarta - Ketua Umum Pro Jokowi ..."
133,07 Des 2019,Amien Rais Tegur Pendukung Zulkifli Hasan di R...,"Liputan6.com, Jakarta - Ketua Dewan Kehormatan..."
134,07 Des 2019,Peran ECommerce Bagi UMKM di Indonesia,"Liputan6.com, Jakarta Maraknya platform berjua..."
135,07 Des 2019,Rakernas PAN Ricuh Amien Rais Istighfar saat L...,"Liputan6.com, Jakarta - Rapat Kerja Nasional (..."


## Simpan ke dalam CSV

In [12]:
df.to_csv('data_liputan6.csv',index=False, encoding='utf-8')

## Simpan ke dalam XLSX

In [13]:
import pandas as pd
data = pd.DataFrame(berita)
writer = pd.ExcelWriter('data_berita.xlsx')
data.to_excel(writer,'Sheet1',index=False)
writer.save()